### word2vec google

osx 直接pip安装

[下载测试数据](http://mattmahoney.net/dc/text8.zip)

In [1]:
import word2vec

In [2]:
#Run word2phrase to group up similar words "Los Angeles" to "Los_Angeles"
#This will create a text8-phrases that we can use as a better input for word2vec. 
#Note that you could easily skip this previous step and use the origial data as input for word2vec.

Train the model using the word2phrase output.
word2vec.word2phrase('/Users/fire/Downloads/text8', '/Users/fire/Downloads/text8-phrases', verbose=True)

Starting training using file /Users/fire/Downloads/text8
Words processed: 17000K     Vocab size: 4399K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


In [4]:
#training
#This generated a text8.bin file containing the word vectors in a binary format.
word2vec.word2vec('/Users/fire/Downloads/text8-phrases', 
                  '/Users/fire/Downloads/text8.bin', 
                  size=100, verbose=True)
                #size：embeding后的单词维度

Starting training using file /Users/fire/Downloads/text8-phrases
Vocab size: 98331
Words in train file: 15857306
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 299.55k  

In [5]:
#Do the clustering of the vectors based on the trained model.
#This created a text8-clusters.txt with the cluster for every word in the vocabulary
word2vec.word2clusters('/Users/fire/Downloads/text8', 
                       '/Users/fire/Downloads/text8-clusters.txt', 
                       100, verbose=True)

Starting training using file /Users/fire/Downloads/text8
Vocab size: 71291
Words in train file: 16718843
Alpha: 0.000002  Progress: 100.03%  Words/thread/sec: 298.98k  

In [8]:
### test
#
model = word2vec.load('/Users/fire/Downloads/text8.bin')
print(model.vocab.shape, model.vectors.shape)
model.vocab

(98331,) (98331, 100)


array(['</s>', 'the', 'of', ..., 'dakotas', 'nias', 'burlesques'],
      dtype='<U78')

In [9]:
model.vectors

array([[ 0.14333282,  0.15825513, -0.13715845, ...,  0.05456942,
         0.10955409,  0.00693387],
       [ 0.12489308, -0.03173497,  0.05842819, ..., -0.05245398,
        -0.00831604, -0.04094807],
       [ 0.24218732,  0.01187136,  0.08873302, ..., -0.0746696 ,
         0.05834081,  0.0571417 ],
       ..., 
       [-0.06582113,  0.10161118,  0.17507532, ...,  0.04679422,
         0.00877832, -0.01720783],
       [-0.0175229 ,  0.00475987,  0.11673692, ..., -0.00197567,
        -0.07617234,  0.02674518],
       [ 0.02870567, -0.04627815,  0.15724808, ...,  0.11611485,
        -0.13422553, -0.0105705 ]])

In [10]:
model['dog'].shape

(100,)

In [11]:
model['dog'][:10]

array([ 0.05836048,  0.11873911, -0.05887308,  0.17603034,  0.09529883,
       -0.05249133,  0.02794671,  0.1341811 ,  0.09122837,  0.00230187])

In [12]:
#We can do simple queries to retreive words similar to "socks" based on cosine similarity:
'''
This returned a tuple with 2 items:

numpy array with the indexes of the similar words in the vocabulary
numpy array with cosine similarity to each word
'''
indexes, metrics = model.cosine('cats')
indexes, metrics

(array([ 3964,  7232,  8755, 10489, 11408,  9017,  7818,  2375,  3336,  5396]),
 array([ 0.88588471,  0.85673789,  0.84536443,  0.82533369,  0.82453982,
         0.82178362,  0.81301288,  0.80753056,  0.8036867 ,  0.79629787]))

In [13]:
model.vocab[indexes]

array(['dogs', 'elephants', 'bees', 'predators', 'goats', 'cows', 'bats',
       'birds', 'horses', 'eggs'],
      dtype='<U78')

In [14]:
model.generate_response(indexes, metrics)

rec.array([('dogs',  0.88588471), ('elephants',  0.85673789),
           ('bees',  0.84536443), ('predators',  0.82533369),
           ('goats',  0.82453982), ('cows',  0.82178362),
           ('bats',  0.81301288), ('birds',  0.80753056),
           ('horses',  0.8036867 ), ('eggs',  0.79629787)], 
          dtype=[('word', '<U78'), ('metric', '<f8')])

In [15]:
model.generate_response(indexes, metrics).tolist()

[('dogs', 0.8858847128930121),
 ('elephants', 0.8567378858448019),
 ('bees', 0.8453644272390131),
 ('predators', 0.825333688468985),
 ('goats', 0.8245398202389704),
 ('cows', 0.8217836200373952),
 ('bats', 0.8130128772507821),
 ('birds', 0.807530559012084),
 ('horses', 0.8036866952392776),
 ('eggs', 0.7962978735121831)]

###  model.analogy

In [16]:

### try king-queen = man-women
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'], n=10)
indexes, metrics

(array([1087, 1145, 7523, 6768,  648,  344, 1335, 8419, 4980, 1826]),
 array([ 0.29280722,  0.27708949,  0.26795348,  0.26184103,  0.26148831,
         0.26143107,  0.26105077,  0.26087758,  0.2580712 ,  0.25800021]))

In [17]:
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'], n=10)
indexes, metrics

(array([1087, 1145, 7523, 6768,  648,  344, 1335, 8419, 4980, 1826]),
 array([ 0.29280722,  0.27708949,  0.26795348,  0.26184103,  0.26148831,
         0.26143107,  0.26105077,  0.26087758,  0.2580712 ,  0.25800021]))

### 聚类

In [27]:
clusters = word2vec.load_clusters('/Users/fire/Downloads/text8-clusters.txt')
clusters[b'dog']  #加b

11

In [23]:
clusters.get_words_on_cluster(90).shape

(1691,)

In [24]:
clusters.get_words_on_cluster(90)[:10]

array([b'secondly', b'insignificant', b'progresses', b'casts',
       b'impractical', b'fortunately', b'attackers', b'filtered',
       b'renaming', b'crafted'], dtype=object)

In [28]:
#We can add the clusters to the word2vec model and generate a response that includes the clusters
model.clusters = clusters

indexes, metrics = model.analogy(pos=['paris', 'germany'], neg=['france'], n=10)
model.generate_response(indexes, metrics).tolist()

[('berlin', 0.327026204998363, 15),
 ('munich', 0.2853918845550195, 44),
 ('vienna', 0.27424945218758845, 12),
 ('moscow', 0.27043351308592767, 74),
 ('st_petersburg', 0.2676503798966319, 61),
 ('leipzig', 0.26643484382369964, 8),
 ('prague', 0.2516699794156753, 85),
 ('dresden', 0.2496050439980815, 71),
 ('bonn', 0.24399744129091042, 8),
 ('frankfurt', 0.2439142560784626, 31)]